# 6. Preprocess Data and Create Labels and Features

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
os.listdir('MP_Data')

['get_out',
 'i_dont_know',
 'i_remember',
 'Please_give_us_privacy',
 'Please_help_me']

In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['get_out', 'i_dont_know', 'i_remember', "Please_give_us_privacy","Please_help_me"])

#Fifty videos worth of data
no_sequences = 50

# Videos are going to be 27 frames in length
sequence_length = 27

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
label_map

{'get_out': 0,
 'i_dont_know': 1,
 'i_remember': 2,
 'Please_give_us_privacy': 3,
 'Please_help_me': 4}

In [7]:
dataset = os.path.join('MP_Data') 


sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(dataset, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(dataset, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
actions.shape[0]

5

In [10]:
np.array(sequences).shape

(250, 27, 1662)

In [11]:
np.array(labels).shape

(250,)

In [12]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [13]:
X.shape,y.shape

((250, 27, 1662), (250, 5))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [15]:
y_test.shape, X_test.shape

((13, 5), (13, 27, 1662))

# 7. Build and Train LSTM Neural Network

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(27,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
8/8 [==============================] - 10s 145ms/step - loss: 1.8141 - categorical_accuracy: 0.1857
Epoch 2/100
8/8 [==============================] - 1s 143ms/step - loss: 1.4438 - categorical_accuracy: 0.3502
Epoch 3/100
8/8 [==============================] - 1s 143ms/step - loss: 0.8353 - categorical_accuracy: 0.7468
Epoch 4/100
8/8 [==============================] - 1s 144ms/step - loss: 0.9541 - categorical_accuracy: 0.7511
Epoch 5/100
8/8 [==============================] - 1s 145ms/step - loss: 1.3769 - categorical_accuracy: 0.4979
Epoch 6/100
8/8 [==============================] - 1s 143ms/step - loss: 1.0965 - categorical_accuracy: 0.5063
Epoch 7/100
8/8 [==============================] - 1s 151ms/step - loss: 0.6229 - categorical_accuracy: 0.7300
Epoch 8/100
8/8 [==============================] - 1s 143ms/step - loss: 0.8759 - categorical_accuracy: 0.9367
Epoch 9/100
8/8 [==============================] - 1s 143ms/step - loss: 2.3877 - categorical_accuracy: 0.3924


8/8 [==============================] - 1s 151ms/step - loss: 3.0212e-05 - categorical_accuracy: 1.0000
Epoch 73/100
8/8 [==============================] - 1s 142ms/step - loss: 2.9287e-05 - categorical_accuracy: 1.0000
Epoch 74/100
8/8 [==============================] - 1s 144ms/step - loss: 2.8471e-05 - categorical_accuracy: 1.0000
Epoch 75/100
8/8 [==============================] - 1s 145ms/step - loss: 2.7683e-05 - categorical_accuracy: 1.0000
Epoch 76/100
8/8 [==============================] - 1s 143ms/step - loss: 2.6936e-05 - categorical_accuracy: 1.0000
Epoch 77/100
8/8 [==============================] - 1s 146ms/step - loss: 2.6245e-05 - categorical_accuracy: 1.0000
Epoch 78/100
8/8 [==============================] - 1s 148ms/step - loss: 2.5537e-05 - categorical_accuracy: 1.0000
Epoch 79/100
8/8 [==============================] - 1s 152ms/step - loss: 2.4845e-05 - categorical_accuracy: 1.0000
Epoch 80/100
8/8 [==============================] - 1s 153ms/step - loss: 2.4214e-05 

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 27, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 27, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596,741
Trainable params: 596,741
Non-trai

# 8. Make Predictions

In [22]:
res = model.predict(X_test)

1/1 [==============================] - 4s 4s/step


In [28]:
res[4]

array([3.9973110e-19, 1.2593890e-12, 1.0000000e+00, 1.9386395e-13,
       2.9349058e-15], dtype=float32)

In [25]:
actions[np.argmax(res[4])]

'i_remember'

In [29]:
actions[np.argmax(y_test[4])]

'i_remember'

# 9. Save Weights

In [30]:
model.save('mani.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [46]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [47]:
yhat = model.predict(X_test)

In [48]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [49]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[17,  0],
        [ 0,  1]],

       [[16,  0],
        [ 0,  2]],

       [[16,  0],
        [ 0,  2]],

       [[13,  0],
        [ 0,  5]],

       [[15,  0],
        [ 0,  3]],

       [[13,  0],
        [ 0,  5]]], dtype=int64)

In [50]:
accuracy_score(ytrue, yhat)

1.0